# Импорт библиотек и создание подключений к БД

In [ ]:
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.engine import URL

from clickhouse_driver import Client

import os

Сonnection string для Microsost SQL Server (где server_name - имя сервера, database_name - имя базы данных):

In [ ]:
connection_string = "DRIVER=SQL Server;SERVER=server_name;DATABASE=database_name;Trusted_Connection=Yes;"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

**NOTE:** Сonnection string для других СУБД: https://docs.sqlalchemy.org/en/20/core/engines.html

Сonnection string для ClickHouse в Yandex Cloud:

In [ ]:
client = Client(host='<FQDN_хоста_ClickHouse>',
                user=os.environ['ch_user'],
                password=os.environ['ch_pwd'],
                port=9440,
                secure=True,
                verify=True,
                ca_certs=os.environ['ch_cert'],
                settings={"use_numpy":True})

**NOTE:**
- где <FQDN_хоста_ClickHouse> - FQDN хоста (о том, как получить FQDN хоста, см. инструкцию: https://cloud.yandex.ru/docs/managed-clickhouse/operations/connect#fqdn)
- Предварительно необходимо создать переменные среды (для Windows: Пуск -> Изменение переменных среды текущего пользователя -> "Создать" переменные среды пользователя):
    - Имя переменной: ch_user, значение переменной: имя_пользователя_БД (см. в консоли ClickHouse в Yandex Cloud)
    - Имя переменной: ch_pwd, значение переменной: пароль_пользователя_БД (см. в консоли ClickHouse в Yandex Cloud)
    - Имя переменной: ch_cert, значение переменной: путь к сертификату (сохраните сертификат (https://storage.yandexcloud.net/cloud-certs/RootCA.pem) в локальную папку и укажите путь к нему)
    
   *после создания переменных среды, нужно перезагрузить (закрыть/открыть) Python*

# Извлечение данных (extract)

Получаем данные из Microsost SQL Server (пример):

In [ ]:
df1 = pd.read_sql(""" SELECT contractid, createdon, department
FROM table21
WHERE createdon >= '2023-10-01' """, engine)

In [ ]:
# выводим первые 5 строк на экран
df1.head(5)

Получаем данные из Excel-файла (пример):

In [ ]:
# Excel-файл содержит 2 столбца: department и region
df2 = pd.read_excel(r'C:\data\Справочник регионов.xlsx')

In [ ]:
# выводим первые 5 строк на экран
df2.head(5)

# Преобразование данных (transform)

Добавляем к выгрузке из MS SQL Server данные из Excel-файла (пример):

In [ ]:
df_final = df1.merge(df2, how='left', on='department')

In [ ]:
# выводим первые 5 строк на экран
df_final.head(5)

# Загрузка данных (load)

Создаём таблицу в ClickHouse в Yandex Cloud **(эту строчку с кодом нужно запустить только 1 раз)** (сейчас закомментировано):

In [ ]:
# client.execute(""" CREATE TABLE db1.tab1(contractid String, createdon Nullable(DateTime64), department Nullable(String), region Nullable(String), _version DateTime64 MATERIALIZED now64())
# ENGINE = ReplacingMergeTree(_version)
# ORDER BY contractid """)

**NOTE:**
- В данном примере создаётся таблица с движком ReplacingMergeTree, он выполняет удаление дублирующихся записей с одинаковым значением ключа сортировки (уникальность строк определяется ORDER BY секцией таблицы, а не PRIMARY KEY, см. подробнее: https://clickhouse.com/docs/ru/engines/table-engines/mergetree-family/replacingmergetree)
- Дедупликация данных производится лишь во время слияний. Слияние происходят в фоне в неизвестный момент времени, на который вы не можете ориентироваться. Некоторая часть данных может остаться необработанной.
- `_version DateTime64 MATERIALIZED now64()` – `_version` это вспомогательный столбец, заполняется автоматически текущей датой и временем

Загружаем данные в созданную таблицу:

In [ ]:
client.insert_dataframe('INSERT INTO db1.tab1 VALUES', df_final)

# Создание датасета в DataLens

Т.к. у движка ReplacingMergeTree дедупликация данных происходит в фоне в неизвестный момент времени, то для получения корректных данных из ClickHouse в DataLens, нужно создать датасет в DataLens через SQL-запрос к источнику (https://cloud.yandex.ru/docs/datalens/operations/dataset/add-data)

**Вариант №1**

SQL-запрос возвращает только данные, которые были были загружены в последний раз:

```
SELECT contractid, createdon, department, region, _version
FROM db1.tab1
WHERE _version = (SELECT MAX(_version) FROM db1.tab1)
```

**Вариант №2**

SQL-запрос возвращает все данные, но для строк с одинаковым ключом – возвращается только самая последняя загруженная строка:

```
SELECT contractid, argMax(createdon, _version) as createdon, argMax(department, _version) as department, argMax(region, _version) as region, max(_version)
FROM db1.tab1
GROUP BY contractid
```